In [1]:
%load_ext autoreload
%autoreload 2
import os
import sys
import gc
import glob
import pandas as pd
import numpy as np
from sklearn.model_selection import KFold
HOME = os.path.expanduser('~')
sys.path.append(f"{HOME}/kaggle/data_analysis/library/")
sys.path.append(f"../py/")
import MS_utils
import utils, ml_utils, kaggle_utils
from feature_engineering import target_encoding
from utils import logger_func
try:
    if not logger:
        logger=logger_func()
except NameError:
    logger=logger_func()

is_debug = 0
key = 'MachineIdentifier'
target = 'HasDetections'
ignore_list = [key, target]

2019-03-06 21:48:01,773 utils 346 [INFO]    [logger_func] start 


In [7]:
agg_list = [
    'f012_SmartScreen'
    ,'f012_AVProductStatesIdentifier'
    ,'f012_CountryIdentifier'
    ,'f012_Census_ActivationChannel'
    ,'f012_Census_OSInstallTypeName'
    ,'f012_AVProductsInstalled'
    ,'f012_Census_GenuineStateName'
    ,'f012_Wdft_RegionIdentifier'
    , 'f009_Engine-Major'
]

# feat_path_list = ml_utils.get_feat_path_list(agg_list, file_key='f0', feat_path='../features/all_features/*.gz')
# feat_path_list = glob.glob('../features/4_winner/f*.gz')
feat_path_list = glob.glob('../features/5_tmp/f*.gz')
p_list = utils.parallel_load_data(path_list=feat_path_list)
tmp_feat = pd.concat(p_list, axis=1)
base = utils.read_df_pkl('../input/base_Av*')[[key, target, 'country_group']]
df_feat = pd.concat([base, tmp_feat], axis=1)

def split_train_test(df, target):
    train = df[~df[target].isnull()]
    test = df[df[target].isnull()]
    return train, test

train, test = ml_utils.split_train_test(df_feat, target)
train.head()

100%|██████████| 3/3 [00:01<00:00,  2.29it/s]


,MachineIdentifier,HasDetections,country_group,f008_RtpStateBitfield,f012_AppVersion-3,f008_OsBuild,f008_LocaleEnglishNameIdentifier,f012_AVProductsInstalled,f012_OSVersion-Sub,f008_OsBuildLab,f012_AvSigVersion-Major,f008_Census_ChassisTypeName,f008_Census_InternalBatteryNumberOfCharges,f008_Census_OSWUAutoUpdateOptionsName,f012_OSVersion-Major,f008_Census_OSEdition,f012_AppVersion-2,f008_Census_FlightRing,f008_Wdft_IsGamer,f008_AppVersion@,f008_Census_OSBuildRevision,f008_OrganizationIdentifier,f012_org-Engine-Major,f012_Census_GenuineStateName,f008_Census_HasOpticalDiskDrive,f008_Census_OEMNameIdentifier,f012_Census_PrimaryDiskTotalCapacity,f012_Census_ActivationChannel,f012_AppVersion-1,f012_Census_OSInstallTypeName,f012_SmartScreen,f012_AvSigVersion-Sub,f008_IsSxsPassiveMode,f012_Census_TotalPhysicalRAM,f012_AVProductStatesIdentifier,f008_Census_OSUILocaleIdentifier,f012_Wdft_RegionIdentifier,f008_Census_InternalPrimaryDisplayResolutionHorizontal,f012_Engine-Sub,f012_CountryIdentifier,f008_CityIdentifier,f008_Census_InternalPrimaryDiagonalDisplaySizeInInches
0,0,0.0,4,7.0,0.0,7.0,112.0,1.0,0.0,307.0,127.0,29.0,29960.0,6.0,17134.0,22.0,0.0,8.0,1.0,0.0,0.0,10.0,0.0,IS_GENUINE,1.0,1231.0,476940.0,Retail,18.0,UUPUpgrade,N,0.0,1.0,4096.0,53447.0,20.0,10.0,450.0,0.0,29,24112.0,143.0
1,1,0.0,6,7.0,1.0,7.0,216.0,1.0,1.0,307.0,126.0,43.0,2.0,6.0,17134.0,22.0,17134.0,5.0,1.0,30.0,2.0,10.0,14600.0,OFFLINE,1.0,1231.0,476940.0,Retail,13.0,IBSClean,N,480.0,1.0,4096.0,53447.0,82.0,8.0,374.0,0.0,93,41795.0,84.0
2,2,0.0,5,7.0,0.0,7.0,199.0,1.0,0.0,307.0,127.0,29.0,29960.0,3.0,17134.0,5.0,0.0,8.0,1.0,0.0,0.0,10.0,0.0,IS_GENUINE,1.0,2230.0,114473.0,OEM:NONSLP,18.0,UUPUpgrade,RequireAdmin,0.0,1.0,4096.0,53447.0,81.0,3.0,930.0,0.0,86,46444.0,173.0
3,3,1.0,2,7.0,0.0,7.0,147.0,1.0,0.0,307.0,127.0,41.0,29960.0,3.0,17134.0,22.0,0.0,8.0,1.0,0.0,0.0,52.0,0.0,IS_GENUINE,1.0,397.0,238475.0,OEM:NONSLP,18.0,UUPUpgrade,ExistsNotSet,0.0,1.0,4096.0,53447.0,115.0,3.0,374.0,0.0,88,62100.0,139.0
4,4,1.0,2,7.0,0.0,7.0,228.0,1.0,0.0,307.0,127.0,47.0,1.0,3.0,17134.0,5.0,0.0,8.0,1.0,0.0,0.0,52.0,0.0,IS_GENUINE,1.0,397.0,476940.0,Retail,18.0,Update,RequireAdmin,0.0,1.0,6144.0,53447.0,82.0,1.0,374.0,0.0,18,76743.0,89.0


In [10]:
#========================================================================
# 組み合わせのTarget Encoding
# TE後に後処理を入れたいときはこちら
# 今はサンプル1,000以下を片方のTEに置き換える処理
#========================================================================
%load_ext autoreload
%autoreload 2
from feature_engineering import target_encoding
from itertools import combinations

kfold = utils.read_pkl_gzip(path='../input/kfold_4fold_ardem.gz')
prefix = 'f003'
feat_cols = [col for col in train.columns if col not in ignore_list and not(col.count('country')) and not(col.count('Internal')) and not(col.count('OsBuildLab')) and not(col.count('Revision')) and not(col.count('RAM')) and not(col.count('Capacity')) ]
level2_list = combinations(feat_cols, 2)
# level2 = ['f012_SmartScreen', 'f009_Engine-Major']
min_sample = 1000

for level2 in level2_list:
    level2 = list(level2)
    level2_name = '-'.join(level2)
    level_list = [level2, level2[0], level2[1]]
    te_list = []
    
    dir_list = ['1_first_valid', 'all_features', '4_winner', '2_second_valid']
    exist_flg = 0
    for dir_name in dir_list:
        f_path = f"../features/{dir_name}/f003_TE_{level2_name}.gz"
        if os.path.exists(f_path):
            exist_flg = 1
    if exist_flg:
        print(level2_name)
        continue
    
    for level in level_list:
    
        feature = target_encoding(
            logger=logger
            , train=train
            , test=test
            , key=key
            , target=target
            , level=level
            , fold_type='self'
            , self_kfold=kfold
            , ignore_list=ignore_list
            , prefix=prefix
        )
        
        if str(type(level)).count('list'):
            feature = pd.Series(feature, name=f"te_{level2_name}")
        else: 
            feature = pd.Series(feature, name=f"te_{level}")
        te_list.append(feature)
                                
    te_sm_en = pd.concat([df_feat[[key, target] + level2]]+te_list, axis=1)
    cnt = df_feat.groupby(level2).size().reset_index()
    cnt.rename(columns={0:'cnt'}, inplace=True)
    df_te = te_sm_en.merge(cnt, how='inner', on=level2)
    
    # min_sample以下はnp.nan
#     df_te.loc[df_te['cnt']<min_sample, f'te_{level2_name}'] = df_te.loc[df_te['cnt']<min_sample, f"te_{level2[0]}"] 
    df_te.loc[df_te['cnt']<min_sample, f'te_{level2_name}'] = -1
    
    for col in [level2_name, level2[0], level2[1]]:
        feature = df_te[f"te_{col}"].values.astype('float32')
        print(feature.shape)
        utils.to_pkl_gzip(obj=feature, path=f'../features/1_first_valid/{prefix}_TE_{col}')

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
f008_RtpStateBitfield-f012_AppVersion-3
f008_RtpStateBitfield-f008_OsBuild
f008_RtpStateBitfield-f008_LocaleEnglishNameIdentifier
f008_RtpStateBitfield-f012_AVProductsInstalled
f008_RtpStateBitfield-f012_OSVersion-Sub
f008_RtpStateBitfield-f012_AvSigVersion-Major
f008_RtpStateBitfield-f008_Census_ChassisTypeName
f008_RtpStateBitfield-f008_Census_OSWUAutoUpdateOptionsName
f008_RtpStateBitfield-f012_OSVersion-Major
f008_RtpStateBitfield-f008_Census_OSEdition
f008_RtpStateBitfield-f012_AppVersion-2
f008_RtpStateBitfield-f008_Census_FlightRing
f008_RtpStateBitfield-f008_Wdft_IsGamer
f008_RtpStateBitfield-f008_AppVersion@
f008_RtpStateBitfield-f008_OrganizationIdentifier
f008_RtpStateBitfield-f012_org-Engine-Major
f008_RtpStateBitfield-f012_Census_GenuineStateName
f008_RtpStateBitfield-f008_Census_HasOpticalDiskDrive
f008_RtpStateBitfield-f008_Census_OEMNameIdentifier
f008_RtpStateBitfield-f012_Census_Ac

2019-03-06 21:53:10,871 utils 277 [INFO]    [target_encoding] Base Train Shape: (8921483, 3) 
2019-03-06 21:53:32,245 utils 304 [INFO]    [target_encoding] 
#========================================================================
# COMPLETE TARGET ENCODING!!
# FEATURE     : TE_HasDetections@['f012_AppVersion-2', 'f012_org-Engine-Major']
# BEFORE LEN  : Train8921483 / Test7853253
# AFTER LEN   : Train8921483 / Test7853253
#======================================================================== 
2019-03-06 21:53:32,366 utils 277 [INFO]    [target_encoding] Base Train Shape: (8921483, 2) 
2019-03-06 21:53:46,094 utils 304 [INFO]    [target_encoding] 
#========================================================================
# COMPLETE TARGET ENCODING!!
# FEATURE     : TE_HasDetections@['f012_AppVersion-2']
# BEFORE LEN  : Train8921483 / Test7853253
# AFTER LEN   : Train8921483 / Test7853253
#======================================================================== 
2019-03-06 21:53:46,214

(16774736,)
(16774736,)
(16774736,)


2019-03-06 21:54:13,647 utils 277 [INFO]    [target_encoding] Base Train Shape: (8921483, 3) 
2019-03-06 21:54:27,128 utils 304 [INFO]    [target_encoding] 
#========================================================================
# COMPLETE TARGET ENCODING!!
# FEATURE     : TE_HasDetections@['f012_AppVersion-2', 'f012_Census_GenuineStateName']
# BEFORE LEN  : Train8921483 / Test7853253
# AFTER LEN   : Train8921483 / Test7853253
#======================================================================== 
2019-03-06 21:54:27,225 utils 277 [INFO]    [target_encoding] Base Train Shape: (8921483, 2) 
2019-03-06 21:54:37,937 utils 304 [INFO]    [target_encoding] 
#========================================================================
# COMPLETE TARGET ENCODING!!
# FEATURE     : TE_HasDetections@['f012_AppVersion-2']
# BEFORE LEN  : Train8921483 / Test7853253
# AFTER LEN   : Train8921483 / Test7853253
#======================================================================== 
2019-03-06 21:54

(16774736,)
(16774736,)
(16774736,)


...

(16774736,)
(16774736,)
(16774736,)


2019-03-07 00:50:00,080 utils 277 [INFO]    [target_encoding] Base Train Shape: (8921483, 3) 
2019-03-07 00:50:19,647 utils 304 [INFO]    [target_encoding] 
#========================================================================
# COMPLETE TARGET ENCODING!!
# FEATURE     : TE_HasDetections@['f012_Engine-Sub', 'f008_CityIdentifier']
# BEFORE LEN  : Train8921483 / Test7853253
# AFTER LEN   : Train8921483 / Test7853253
#======================================================================== 
2019-03-07 00:50:19,740 utils 277 [INFO]    [target_encoding] Base Train Shape: (8921483, 2) 
2019-03-07 00:50:33,684 utils 304 [INFO]    [target_encoding] 
#========================================================================
# COMPLETE TARGET ENCODING!!
# FEATURE     : TE_HasDetections@['f012_Engine-Sub']
# BEFORE LEN  : Train8921483 / Test7853253
# AFTER LEN   : Train8921483 / Test7853253
#======================================================================== 
2019-03-07 00:50:33,788 utils

(16774736,)
(16774736,)
(16774736,)


2019-03-07 00:51:04,193 utils 277 [INFO]    [target_encoding] Base Train Shape: (8921483, 3) 
2019-03-07 00:51:22,053 utils 304 [INFO]    [target_encoding] 
#========================================================================
# COMPLETE TARGET ENCODING!!
# FEATURE     : TE_HasDetections@['f012_CountryIdentifier', 'f008_CityIdentifier']
# BEFORE LEN  : Train8921483 / Test7853253
# AFTER LEN   : Train8921483 / Test7853253
#======================================================================== 
2019-03-07 00:51:22,174 utils 277 [INFO]    [target_encoding] Base Train Shape: (8921483, 2) 
2019-03-07 00:51:34,925 utils 304 [INFO]    [target_encoding] 
#========================================================================
# COMPLETE TARGET ENCODING!!
# FEATURE     : TE_HasDetections@['f012_CountryIdentifier']
# BEFORE LEN  : Train8921483 / Test7853253
# AFTER LEN   : Train8921483 / Test7853253
#======================================================================== 
2019-03-07 00:5

(16774736,)
(16774736,)
(16774736,)


In [79]:
prefix = 'f003_TE'
save_list = ['f012_SmartScreen-f009_Engine-Major', 'te_f012_SmartScreen', 'te_f009_Engine-Major']
for col in save_list:
    feature = df_te[col].values.astype('float32')
    col = col.replace('te_', '')
    utils.to_pkl_gzip(obj=feature, path=f'../features/1_first_valid/{prefix}_{col}')

(16771446,)
(16771446,)
(16771446,)


In [3]:
%load_ext autoreload
%autoreload 2
from feature_engineering import target_encoding

group_list = train['country_group'].unique()
valid_list = []
for group in group_list:
    tmp = train[train['country_group']==group][[key, target]]
    valid_list.append(tmp)
    
kfold = utils.get_kfold(fold_seed=1208, valid_list=valid_list, key=key, target=target)
prefix = 'f003'

for col in train.columns:
    if col in ignore_list or col.count('country_'):
        continue
        
    feature = target_encoding(
        logger=logger
        , train=train
        , test=test
        , key=key
        , target=target
        , level=col
        , fold_type='self'
        , self_kfold=kfold
        , ignore_list=ignore_list
        , prefix=prefix
    )
    utils.to_pkl_gzip(obj=feature, path=f'../features/1_first_valid/{prefix}_TE_{col}')

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


2019-03-03 23:07:27,187 utils 277 [INFO]    [target_encoding] Base Train Shape: (8921483, 2) 
2019-03-03 23:08:05,697 utils 304 [INFO]    [target_encoding] 
#========================================================================
# COMPLETE TARGET ENCODING!!
# FEATURE     : TE_HasDetections@['country_group']
# BEFORE LEN  : Train8921483 / Test7853253
# AFTER LEN   : Train8921483 / Test7853253
#======================================================================== 
2019-03-03 23:08:11,076 utils 277 [INFO]    [target_encoding] Base Train Shape: (8921483, 2) 
2019-03-03 23:08:53,405 utils 304 [INFO]    [target_encoding] 
#========================================================================
# COMPLETE TARGET ENCODING!!
# FEATURE     : TE_HasDetections@['country_os_group']
# BEFORE LEN  : Train8921483 / Test7853253
# AFTER LEN   : Train8921483 / Test7853253
#======================================================================== 
2019-03-03 23:08:57,631 utils 277 [INFO]    [target_e